In [ ]:
%pip install langgraph langchain langchain_openai langchain_community
%pip install -q langfuse

In [22]:
import os, getpass, requests
import sqlite3
from pathlib import Path
import datetime

from typing import TypedDict, List, Optional, Annotated
from IPython.display import Image, display

from langfuse import Langfuse
from langfuse.callback import CallbackHandler

from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import ToolNode
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

In [23]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")
_set_env("LANGFUSE_PUBLIC_KEY")
_set_env("LANGFUSE_SECRET_KEY")
_set_env("LANGFUSE_HOST")
_set_env("LANGCHAIN_API_KEY")
_set_env("LANGCHAIN_TRACING_V2")
_set_env("UNSPLASH_ACCESS_KEY")
_set_env("UNSPLASH_SECRET")
_set_env("GOOGLE_API_KEY")
_set_env("GOOGLE_CSE_ID")

langfuse = Langfuse(
    secret_key=os.environ["LANGFUSE_SECRET_KEY"],
    public_key=os.environ["LANGFUSE_PUBLIC_KEY"],
    host=os.environ["LANGFUSE_HOST"]
)

langfuse_handler = CallbackHandler()

DB_PATH = Path("./articles.sqlite")
conn = sqlite3.connect(DB_PATH)
memory = SqliteSaver(conn)

In [24]:
class Draft(TypedDict):
    title: str
    body: str
    images: List[str]

def update_draft(left: Draft | None, right: Draft | None) -> Draft:
    if left is None:
        return right
    if right is None:
        return left
    draft = Draft(
        title=right["title"] if right["title"] != "" else left["title"] if left else "",
        body=right["body"] if right["body"] != "" else left["body"] if left else "",
        images=right["images"] if right["images"] != [] else left["images"] if left else []
    )
    return draft

class Source(TypedDict):
    title: str
    url: str
    content: str
    score: float
    valid: bool
    reason: Optional[str]

class State(MessagesState):
    prompt: str
    topic: str
    sources: Optional[List[Source]]
    draft: Annotated[Draft, update_draft]
    article: Optional[str]
    feedback_title: Optional[str]
    feedback_body: Optional[str]
    feedback_images: Optional[List[str]]

In [25]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7,
    streaming=True,
    verbose=True,
    callbacks=[langfuse_handler],
)

tavily = TavilySearchResults(max_results=5)

In [26]:
def get_user_prompt_node(state: State) -> State:
    '''
    Get the user prompt node.
    '''
    prompt = input(
        "Inserisci il prompt per l'articolo: "
    )
    return { "prompt": prompt }

In [27]:
def extract_topic_node(state: State) -> State:
    '''
    Extract the topic from the user prompt.
    '''
    
    prompt = f'''
    The user provided the following input:

    "{state["prompt"]}"
    Your task is to extract the main topic of the article that the user wants to write.
    The topic should be a single word or a short phrase that summarizes the main idea of the article.

    For example, if the user prompt is "How to train a dog", the topic would be "dog training".

    If the user prompt is "The history of the internet", the topic would be "internet history".

    Only return the topic.
    '''

    print(f"Prompt: {prompt}")

    response = llm.invoke(prompt)

    print(f"Response: {response}")
    
    topic = response.content.strip()
    return { "topic": topic }

In [28]:
def search_sources_node(state: State) -> State:
    '''
    Use Tavily to search for high-quality sources based on the topic.
    Filters results with score > 0.7 and keeps structured data.
    '''
    
    results = tavily.invoke({ "query": state["topic"] })

    formatted_results: List[Source] = [
        Source(
            title=r.get("title", ""),
            url=r.get("url", ""),
            content=r.get("content", ""),
            score=r.get("score", 0),
            valid=r.get("score", 0) > 0.7,
            reason=None
        )
        for r in results if r.get("score", 0) > 0.7
    ]

    return { "sources": formatted_results }

In [29]:
def verify_sources_node(state: State) -> State:
    '''
    Use the LLM to verify the quality and reliability of each source based on its content and URL.
    For each source, return if it is reliable and usable for writing the article, and if not, provide a reason why.
    '''
    
    sources = state["sources"]
    
    verified_sources = []
    for source in sources:
        url = source["url"]
        content = source["content"]
        
        prompt = f'''
        You are a reliability and fact-checking assistant. Below is a source with its content:

        URL: {url}
        Content: {content}

        Question: Based on the content and the URL, is this source reliable and usable for writing an article? 

        Important: Do **NOT** automatically dismiss a source as unreliable simply because the event it discusses is in the future. 
        Assess the source based on the quality and consistency of the content, the reputation of the outlet, and any references to authoritative data or expert opinions. 
        If the content is speculative or based on rumors, you may indicate that it is unreliable. 
        However, future events should not be ruled out solely because they have not occurred yet.
        
        Please provide a "YES" or "NO" answer. If "NO", explain why it is not reliable or usable.

        Use the following format:
        Reliable: <YES/NO>
        Explanation: <explanation>
        
        For example:
        Reliable: YES
        Explanation: The source is from a reputable news outlet and provides well-cited information. The content is well-researched and aligns with known facts.
        '''
        
        response = llm.invoke(prompt)
        result = response.content.strip()

        try:
            reliable_line = next(line for line in result.split("\n") if line.startswith("Reliable:"))
            explanation_line = next(line for line in result.split("\n") if line.startswith("Explanation:"))

            reliable = "YES" in reliable_line
            explanation = explanation_line.split(":")[1].strip()

        except StopIteration:
            reliable = False
            explanation = "Failed to extract reliable information from the source."

        verified_sources.append({
            "title": source["title"],
            "url": url,
            "content": content,
            "reliable": reliable,
            "reason": explanation if not reliable else None,
        })

    print("Verified sources: ")
    for source in verified_sources:
        print(f"Title: {source['title']}, Reliable: {source['reliable']}, Reason: {source['reason']}")
        
    usable_sources = [source for source in verified_sources if source["reliable"]]
    
    return { "sources": usable_sources }

In [30]:
def generate_title_node(state: State) -> State:
    '''
    Generate a title for the article based on the topic and sources.
    '''
    
    prompt = f'''
    You are an expert article writer. Based on the following topic and sources, generate a catchy and clickbait title for the article.
    The title MUST BE IN ITALIAN🇮🇹🇮🇹🇮🇹🇮🇹🇮🇹.

    Topic: {state["topic"]}
    Sources: {state["sources"]}

    The title should be engaging and reflect the content of the article. Exagerate the topic and make it sound interesting.
    The title should be no more than 10 words long and should not contain any special characters or numbers.
    For example, if the topic is "dog training" and the sources are about dog training techniques, a possible title could be "The Ultimate Guide to Dog Training: Tips and Tricks".
    Only return the title.
    Do not include any additional text or explanations.
    '''

    response = llm.invoke(prompt)
    
    title = response.content.strip()
    return {
        "draft": {
            "title": title,
            "body": "",
            "images": []
        }
    }

In [ ]:
def generate_draft_node(state: State) -> State:
    '''
    Generate a draft for the article based on the topic and sources.
    '''

    urls = [source["url"] for source in state["sources"] if source["url"]]
    sources = "\n".join(urls)
    
    prompt = f'''
    Write a detailed article about the topic: "{state["topic"]}". 

    Use **ONLY** the exact information contained in the following sources, and do not add anything that is not directly supported by them. 
    The article should strictly reflect the facts as they are presented in the sources without any additional assumptions or unverified details:
    {sources}

    Make sure to mention the exact dates and details from the sources, without adding any speculative or external information. 
    The tone should be formal, informative, and objective.
    '''

    response = llm.invoke(prompt)
    body = response.content.strip()
    return {
        "draft": {
            "title": "",
            "body": body,
            "images": []
        }
    }

In [32]:
def generate_images_node(state: State) -> State:
    """
    Cerca su Google Custom Search Engine le prime 3 immagini relative a state['topic'],
    le visualizza inline e aggiorna state['draft']['images'] con gli URL.
    """
    prompt= f'''
    Sei un agente di ricerca di immagini.
    Devi generare una query da passare all'API di Google per 
    cercare immagini relative al seguente argomento: "{state["topic"]}"
    basandoti sul feedback dell'utente "{state["feedback_images"]}".
    Se il feedback è vuoto o non esiste ignoralo.
    Restituisci solo la query, senza alcun testo aggiuntivo.
    Esempio:
    Topic: "Finale del Mondiale 2022"
    Feedback: ""
    Query: "Immagini finale del Mondiale di calcio 2022"
    Altro esempio:
    Topic: "Elezioni presidenziali USA 2024"
    Feedback: "Voglio immagini di Trump"
    Query: "Immagini Trump elezioni presidenziali USA 2024"
    '''
    response = llm.invoke(prompt)
    query = response.content.strip()
    print(f"Query: {query}")
    api_key = os.getenv("GOOGLE_API_KEY")
    cx = os.getenv("GOOGLE_CSE_ID")
    if not api_key or not cx:
        raise RuntimeError("Setta GOOGLE_API_KEY e GOOGLE_CSE_ID nell'ambiente")
    endpoint = "https://www.googleapis.com/customsearch/v1"

    params = {
        "key": api_key,
        "cx": cx,
        "q": query,
        "searchType": "image",
        "num": 3,
        "imgSize": "medium",
        "cr": "countryIT | countryUS",
        "gl": "it",
        "hl": "it",
        "imgColorType": "color",
    }

    resp = requests.get(endpoint, params=params)
    resp.raise_for_status()
    items = resp.json().get("items", [])

    urls: List[str] = [item["link"] for item in items]

    for url in urls:
        display(Image(url=url))

    return {
        "draft": {
            "title": "",
            "body": "",
            "images": urls
        }
    }

In [33]:
def feedback_title_router(state: State) -> State:
    '''
    Mostra il titolo generato e raccoglie feedback per il titolo.
    '''
    title = state.get('draft', {}).get('title', '')
    print(f'Titolo generato:\n{title}\n')
    state["feedback_title"] = input('✅ Feedback per il titolo: ')
    prompt= f'''
    Devi decidere se chiamare il tool "generate_title" o andare al nodo successivo "render_article"
    sulla base del feedback dell'utente "{state["feedback_title"]}". Se il feedback è negativo significa che il titolo non è soddisfacente e quindi
    il tool "generate_title" deve essere chiamato. Se il feedback è positivo significa che il titolo è soddisfacente e quindi
    bisogna andare al nodo successivo "render_article".
    Se il feedback è vuoto o non esiste ignoralo e vai avanti.
    Restituisci solo il nome del tool, senza alcun testo aggiuntivo.
    '''
    response = llm.invoke(prompt)
    return response.content.strip()

def feedback_body_router(state: State) -> State:
    '''
    Mostra il corpo generato e raccoglie feedback per il corpo.
    '''
    body = state.get('draft', {}).get('body', '')
    print(f'Corpo generato:\n{body}\n')
    state["feedback_body"] = input('✅ Feedback per il corpo: ')
    prompt= f'''
    Devi decidere se chiamare il tool "generate_draft" o andare al nodo successivo "render_article"
    sulla base del feedback dell'utente "{state["feedback_body"]}". Se il feedback è negativo significa che il corpo non è soddisfacente e quindi
    il tool "generate_draft" deve essere chiamato. Se il feedback è positivo significa che il corpo è soddisfacente e quindi
    bisogna andare al nodo successivo "render_article".
    Se il feedback è vuoto o non esiste ignoralo e vai avanti.
    Restituisci solo il nome del tool, senza alcun testo aggiuntivo.
    '''
    response = llm.invoke(prompt)
    return response.content.strip()

def feedback_images_router(state: State) -> State:
    '''
    Elenca le immagini generate e raccoglie feedback per le immagini.
    '''
    images = state.get('draft', {}).get('images', [])
    print(f'Immagini generate:')
    for image in images:
        display(Image(url=image))
    state["feedback_images"] = input('✅ Feedback per le immagini: ')
    prompt= f'''
    Devi decidere se chiamare il tool "generate_images" o andare al nodo successivo "render_article"
    sulla base del feedback dell'utente "{state["feedback_images"]}". Se il feedback è negativo significa che le immagini non sono soddisfacenti e quindi
    il tool "generate_images" deve essere chiamato. Se il feedback è positivo significa che le immagini sono soddisfacenti e quindi
    bisogna andare al nodo successivo "render_article".
    Se il feedback è vuoto o non esiste ignoralo e vai avanti.
    Restituisci solo il nome del tool, senza alcun testo aggiuntivo.
    '''
    response = llm.invoke(prompt)
    return response.content.strip()

In [34]:
def render_article_node(state: State) -> State:
    """
    Combina title, body e images in un unico articolo Markdown,
    rifinisce la formattazione con l'LLM e lo visualizza in output.
    """
    draft = state.get("draft", {"title": "", "body": "", "images": []})
    title = draft["title"]
    body = draft["body"]
    images = draft["images"]

    images_md = "\n".join(f"![{title}]({url})" for url in images)

    prompt = f"""
        Unisci questi elementi in un unico articolo Markdown ben formattato:
        Titolo:
        {title}

        Corpo:
        {body}

        Immagini (Markdown già pronto):
        {images_md}

        Un'immagine deve essere messa all'inizio, seguita dal titolo e poi dalle
        altre due immagini mescolate in maniera fluida al corpo, con un'impostazione
        da articolo online.
        Restituisci solo il testo Markdown finale.
        """
    response = llm.invoke(prompt)
    article_md = response.content.strip()

    return { "article": article_md }

In [35]:
def save_article_node(state: State) -> State:
    db_path = state["db_path"]
    conn    = sqlite3.connect(db_path)

    topic   = state.get("topic", "")
    article = state.get("article", "")
    ts      = datetime.utcnow().isoformat()

    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS articles (
            id         INTEGER PRIMARY KEY AUTOINCREMENT,
            topic      TEXT NOT NULL,
            article    TEXT NOT NULL,
            created_at TEXT NOT NULL
        )
    """)
    cursor.execute(
        "INSERT INTO articles (topic, article, created_at) VALUES (?, ?, ?)",
        (topic, article, ts)
    )
    conn.commit()
    conn.close()

    return state

In [46]:
tools_title = [generate_title_node, render_article_node]
tools_draft = [generate_draft_node, render_article_node]
tools_images = [generate_images_node, render_article_node]
builder = StateGraph(State)
builder.add_node("get_user_prompt", get_user_prompt_node)
builder.add_node("extract_topic", extract_topic_node)
builder.add_node("search_sources", search_sources_node)
builder.add_node("verify_sources", verify_sources_node)
builder.add_node("generate_title", generate_title_node)
builder.add_node("title_feedback", ToolNode(tools_title))
builder.add_node("generate_draft", generate_draft_node)
builder.add_node("body_feedback", ToolNode(tools_draft))
builder.add_node("generate_images", generate_images_node)
builder.add_node("images_feedback", ToolNode(tools_images))
builder.add_node("render_article", render_article_node)
builder.add_node("save_article", save_article_node)

builder.add_edge(START, "get_user_prompt")
builder.add_edge("get_user_prompt", "extract_topic")
builder.add_edge("extract_topic", "search_sources")
builder.add_edge("search_sources", "verify_sources")
builder.add_edge("verify_sources", "generate_title")
builder.add_edge("verify_sources", "generate_draft")
builder.add_edge("verify_sources", "generate_images")
builder.add_edge("generate_title", "title_feedback")
builder.add_conditional_edges("title_feedback", feedback_title_router)
builder.add_edge("generate_draft", "body_feedback")
builder.add_conditional_edges("body_feedback", feedback_body_router)
builder.add_edge("generate_images", "images_feedback")
builder.add_conditional_edges("images_feedback", feedback_images_router)
builder.add_edge("render_article", "save_article")
builder.add_edge("save_article", END)

graph = builder.compile().with_config(
    callbacks=[langfuse_handler],
    memory=memory,
)

In [51]:
display(Image(graph.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [43]:
graph.invoke({})

Prompt: 
    The user provided the following input:

    "Scrivi un articolo sulla finale dei Mondiali 2022"
    Your task is to extract the main topic of the article that the user wants to write.
    The topic should be a single word or a short phrase that summarizes the main idea of the article.

    For example, if the user prompt is "How to train a dog", the topic would be "dog training".

    If the user prompt is "The history of the internet", the topic would be "internet history".

    Only return the topic.
    
Response: content='Finale Mondiali 2022' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_55d88aaf2f'} id='run-8e395a9c-9159-4149-8c0e-b5c3f6e77a99-0'
Verified sources: 
Title: Finale del campionato mondiale di calcio 2022 - Wikipedia, Reliable: True, Reason: None
Title: Finale Mondiali 2022, vince l'Argentina: batte la Francia ai rigori, Reliable: True, Reason: None
Title: 2022 FIFA World Cup 

```markdown
![Argentina Trionfa: La Finale Mondiale Più Epica di Sempre](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/ARG_Line-up_-_ARG_vs_MEX_for_2022_FIFA_WC.jpg/260px-ARG_Line-up_-_ARG_vs_MEX_for_2022_FIFA_WC.jpg)

# Argentina Trionfa: La Finale Mondiale Più Epica di Sempre

**Finale Mondiali 2022: Un Resoconto Dettagliato**

La finale del Campionato mondiale di calcio 2022 si è svolta il 18 dicembre 2022 presso il Lusail Iconic Stadium di Lusail, Qatar. Questo evento ha segnato la conclusione del torneo FIFA World Cup 2022, in cui l'Argentina ha affrontato la Francia. È stata una partita che ha catturato l'attenzione di milioni di appassionati di calcio in tutto il mondo, offrendo un'esperienza emozionante e spettacolare.

![Argentina Trionfa: La Finale Mondiale Più Epica di Sempre](https://static.sky.it/editorialimages/6148ab643ff2c175883718f683679ff439ca0c08/skysport/it/calcio/mondiali/classifica-marcatori-mondiali-2022-qatar/messi_mbappe.jpg?im=Resize,width=375)

**Contesto e Squadre Finaliste**

L'Argentina, guidata dal loro capitano Lionel Messi, è arrivata a questa finale dopo una serie impressionante di prestazioni nel torneo. L'allenatore Lionel Scaloni ha saputo guidare la squadra attraverso le fasi eliminatorie con abilità tattica e spirito di squadra. Dall'altra parte, la Francia, campione del mondo in carica, ha cercato di difendere il titolo vinto nel 2018. Sotto la guida dell'allenatore Didier Deschamps, la squadra francese ha dimostrato un'eccezionale profondità di rosa e talento.

**La Partita**

La finale tra Argentina e Francia è iniziata con un'intensità palpabile. L'Argentina ha preso il comando nel primo tempo grazie ai gol di Lionel Messi su rigore al 23° minuto e Ángel Di María al 36° minuto, portandosi avanti di due reti. La Francia, tuttavia, non è rimasta a guardare. Kylian Mbappé ha segnato due gol in rapida successione all'80° e all'81° minuto, portando il match sul 2-2 e mandando la partita ai tempi supplementari.

Nei supplementari, l'Argentina è tornata in vantaggio con un altro gol di Messi al 108° minuto. Tuttavia, la resilienza della Francia è stata evidente quando Mbappé ha completato la sua tripletta personale al 118° minuto su rigore, portando il punteggio sul 3-3 e forzando la partita ai calci di rigore.

![Argentina Trionfa: La Finale Mondiale Più Epica di Sempre](https://entry.motorsport.tv/images/episode/id130529/contentCarousel/15ed06b71b79368af28841d51da8d677fe7445f1-549-549.jpg?w=320)

**I Calci di Rigore**

La tensione è culminata nei calci di rigore, una situazione spesso imprevedibile e carica di adrenalina. L'Argentina è riuscita a prevalere con un punteggio di 4-2 nei rigori, grazie anche alla prestazione decisiva del portiere Emiliano Martínez, che ha parato il tiro di Kingsley Coman. L'errore decisivo di Aurélien Tchouaméni ha consegnato la vittoria all'Argentina, sancendo così il terzo titolo mondiale della storia dell'Albiceleste.

**Conclusione**

La vittoria dell'Argentina nella finale del Mondiale 2022 è stata celebrata come un trionfo storico, soprattutto per Lionel Messi, che ha realizzato il suo sogno di vincere un mondiale e ha cementato ulteriormente la sua eredità nel calcio mondiale. La partita è stata un esemplare di drammaticità sportiva e ha dimostrato l'imprevedibilità e la bellezza del calcio a livello mondiale.
```

{'messages': [],
 'prompt': 'Scrivi un articolo sulla finale dei Mondiali 2022',
 'topic': 'Finale Mondiali 2022',
 'sources': [{'title': 'Finale del campionato mondiale di calcio 2022 - Wikipedia',
   'url': 'https://it.wikipedia.org/wiki/Finale_del_campionato_mondiale_di_calcio_2022',
   'content': "La finale del campionato mondiale di calcio 2022 si è disputata il 18 dicembre 2022 allo stadio Iconico di Lusail tra l'Argentina e la Francia. [...] Considerata da alcuni osservatori la migliore finale nella storia dei Mondiali,[3][4] nonché uno degli incontri calcistici più spettacolari di tutti i tempi,[5][6] la sfida è stata vinta dall'Argentina per 4-2 ai tiri di rigore, dopo il 2-2 dei tempi regolamentari e il 3-3 maturato nei tempi supplementari, per un risultato complessivo di 7-5:[7] la nazionale sudamericana si aggiudicò così la sua terza Coppa del Mondo FIFA, a 36 anni di distanza dal titolo precedente. [...] Informazioni generali\nSport | Calcio\nCompetizione | Campionato mond